<h1> Question Answering - Code Generation </h1>
<br>

Before starting, please make sure this notebook is using **conda_python3** kernel from the top right!

To run this notebook, go to **Cell -> Run All**. Read the comments in the notebook and inspect the output of each cell.

### Introduction

In this notebook, you will create a Q&A application using Bedrock. Following is the flow.

1. Customer asks a question in natural language.
2. Use Text-to-Text LLM to generate SQL query for the question. LLM will use the schema of the tables containing retail website data. This will be passed as one of the prompt input variables.
3. Execute the query against the website data stored in Amazon RDS Postgres database.
4. Use Text-to-Text LLM to interpret the query results in natural language.
5. Display the answer to the question.

<br><br>


![Code Generation](../images/sql-generation.png)


### Install required dependencies

**Important:** You may see an error or a warning that "you may need to restart the kernel" from the following cell. **Ignore** and proceed with the next cells. 

In [ ]:
%pip install --quiet --no-build-isolation --upgrade \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" \
    "langchain==0.0.309" \
    "psycopg2-binary==2.9.9"

<h3> Import required packages </h3>

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock
import psycopg2

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3>

In [ ]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#### Get the name of the S3 bucket created from CloudFormation

In [ ]:
s3 = boto3.client('s3')

s3_bucket = None
response = s3.list_buckets()
for bucket in response['Buckets']:
    if 'reinvent-retails-bucket' in bucket['Name']: 
        s3_bucket = bucket['Name']

print(s3_bucket)

#### Read postgres database schema

Typically, all the data from web applications are stored in a database. In our case, all the data from our re:Invent retails web application (like products, orders, customer reviews, product ratings etc.) are stored in RDS Postgres database. You can easily get the schema of the tables with the [pg_dump](https://www.postgresql.org/docs/current/app-pgdump.html) tool. We have already taken a dump of the table schemas from the RDS database and uploaded it to S3. Let's read this schema. 

In [ ]:
s3_response = s3.get_object(Bucket=s3_bucket, Key="data/schema-mysql.sql")
schema = s3_response['Body'].read().decode("utf-8")

# Printing schemas of all the tables used in our web application
print(schema)

### Prompt template

<p>Using this prompt template we are asking the LLM to generate an SQL query based on the Postgres database schema. Notice that we pass 7 rules for the LLM to adhere to while constructing the query. Read the prompt instructions carefully. </p>

In [ ]:
prompt_template = """
            Human: Create an Postgres SQL query for a retail website to answer the question keeping the following rules in mind: 
            
            1. Database is implemented in Postgres SQL.
            2. Follow the Postgres syntax carefully while generating the query.
            3. Enclose the query in <query></query>. 
            4. Use "like" and upper() for string comparison on both left hand side and right hand side of the expression. For example, if the query contains "jackets", use "where upper(product_name) like upper('%jacket%')". 
            5. If the question is generic, like "where is mount everest" or "who went to the moon first", then do not generate any query in <query></query> and do not answer the question in any form. Instead, mention that the answer is not found in context.
            6. If the question is not related to the schema, then do not generate any query in <query></query> and do not answer the question in any form. Instead, mention that the answer is not found in context.  
            7. If the question is asked in a language other than English, convert the question into English before constructing the SQL query. The string and varchar fields stored in the database are always in English.  

            <schema>
                {schema}
            </schema>

            Question: {question}

            Assistant:
            
            """

#### Ask a sample question in natural language

In [ ]:
question = "How many products do you have in your store?"

#### Pass this question to prompt template

In [ ]:
prompt_vars = PromptTemplate(template=prompt_template, input_variables=["question","schema"])

#### Now lets call the Bedrock LLM to generate SQL query

In [ ]:
#Initialize Claude Anthropic LLM
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=boto3_bedrock)

# Pass question and postgres schema of the web application
prompt = prompt_vars.format(question=question, schema=schema)

# Print response
llm_response = llm(prompt)
print(llm_response)

#### As you can see from the above llm_response, the query is embedded inside \<query\>\<\/query\> tags. Let's create a function retrieve the query inside the \<query\> tags. 

In [ ]:
def extract_strings_recursive(test_str, tag):
    # finding the index of the first occurrence of the opening tag
    start_idx = test_str.find("<" + tag + ">")
 
    # base case
    if start_idx == -1:
        return []
 
    # extracting the string between the opening and closing tags
    end_idx = test_str.find("</" + tag + ">", start_idx)
    res = [test_str[start_idx+len(tag)+2:end_idx]]
 
    # recursive call to extract strings after the current tag
    res += extract_strings_recursive(test_str[end_idx+len(tag)+3:], tag)
 
    return res

#### Get the query using the extract_strings_recursive function we created above

In [ ]:
query = ""

if "<query>".upper() not in llm_response.upper():
    is_query_generated  = False
else:
    is_query_generated  = True
    query = extract_strings_recursive(llm_response, "query")[0]
    print("Query generated by LLM: \n" +query)

### Execute the query

Now, let's connect to the RDS database and run this LLM-generated query. 

In [ ]:
# Initialize secrets manager
secrets = boto3.client('secretsmanager')

sm_response = secrets.get_secret_value(SecretId='postgresdb-secrets')

database_secrets = json.loads(sm_response['SecretString'])

# Get secrets from the secrets manager
dbhost = database_secrets['host']
dbport = database_secrets['port']
dbuser = database_secrets['username']
dbpass = database_secrets['password']
dbname = database_secrets['name']

# Connect to PostgreSQL database
dbconn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, database=dbname, connect_timeout=10)
dbconn.set_session(autocommit=True)
cursor = dbconn.cursor()

#### Execute the LLM-generated query in Amazon RDS

In [ ]:
# Execute the extracted query
cursor.execute(query)

query_result = cursor.fetchall()
dbconn.close()

resultset = ''
if len(query_result) > 0:
    for x in query_result:
        resultset = resultset + ''.join(str(x)) + "\n"

print("Query result: \n" +resultset)

#### Now, lets create another prompt template to interpret the query results. 

This prompt template defines rules while describing query result. This is the final result that will be seen by the user as an answer to their question. Idea is to derive natural language answer for a natural language question. 

In [ ]:
prompt_template = """

        Human: This is a Q&A application. We need to answer questions asked by the customer at an e-commerce store. 
        The question asked by the customer is {question}
        
        We ran an SQL query in our database to get the following result. 

        <resultset>
            {resultset}
        </resultset>

        Summarize the above result and answer the question asked by the customer keeping the following rules in mind: 
        
        1. Don't make up answers if <resultset></resultset> is empty or none. Instead, answer that the item is not available based on the question.
        2. Mask the PIIs phone, email and address if found the answer with "<PII masked>"
        3. Don't say "based on the output" or "based on the query" or "based on the question" or something similar.  
        4. Keep the answer concise. 
        5. Don't give an impression to the customer that a query was run. Instead, answer naturally. 

        Assistant:

        """

#### Create prompt and invoke LLM to interpret results retrieved from the database

In [ ]:
prompt_vars = PromptTemplate(template=prompt_template, input_variables=["question","resultset"])

# Pass question and result set to prompt template
prompt = prompt_vars.format(question=question, resultset=resultset)

# Call LLM to interpret query result
describe_query_result = llm(prompt)

print("Question asked by the user: \n")
print(question + "\n\n")
print("Answer interpreted by LLM: \n")
print_ww(describe_query_result)

<h3> You've successfully created a Q&A application using SQL generation with Amazon Bedrock!</h3>

Please stop the notebook kernel before proceeding with **Kernel -> Interrupt**.

#### Now, let's integrate this feature into our retail web application. Please go back to Workshop Studio and follow the instructions to build this feature using your Cloud9 IDE.